In [1]:
import os
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]=""

# 📂DATA : 3 Classes

In [31]:
fold = 3

In [32]:
path = "/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv"
data_feature = pd.read_csv(path)
print(data_feature.shape)
print("-"*100)
print(f"All Fold : {set(data_feature.fold)}")
## Split Train data Set
feature_train = data_feature[data_feature["fold"]!=fold].reset_index(drop=True)
print(f"Train Set : Fold ==> {set(feature_train.fold)}")
print("Train = ", feature_train.shape)
## Split Valid data Set
feature_test = data_feature[data_feature["fold"]==fold].reset_index(drop=True)
print(f"Valiadtion Set : Fold ==> {set(feature_test.fold)}")
print("Validation = ", feature_test.shape)
## Print DataFrame
feature_train.head()

(51, 9)
----------------------------------------------------------------------------------------------------
All Fold : {1, 2, 3}
Train Set : Fold ==> {1, 2}
Train =  (34, 9)
Valiadtion Set : Fold ==> {3}
Validation =  (17, 9)


,Unnamed: 0,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
0,0,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
1,1,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
2,2,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
3,3,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
4,4,NBL23,NBL,Normal,1,0.0,66.5,20.3,13.0


In [33]:
X_train = feature_train[['MCV','MCH','Hb']]
y_train = feature_train["subclass"]
print(X_train.shape)
print(y_train.shape)
print(y_train[0])

(34, 3)
(34,)
No_Splenectomy


In [34]:
print(len(list(set(y_train))))
print(set(y_train))

3
{'Normal', 'No_Splenectomy', 'Splenectomy'}


## 🩸 Parameter:  Logistic Regression

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [36]:
LR = LogisticRegression(random_state = 1)

LRparam_grid = {
    'C' : [0.001, 0.01, 0.1, 1],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,500,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 1, cv=10, n_jobs = -1)
# fitting the model for grid search 
best_LR = LR_search.fit(X_train, y_train)

Fitting 10 folds for each of 160 candidates, totalling 1600 fits


In [37]:
# view the results as a pandas DataFrame
best_LRdf = pd.DataFrame(best_LR.cv_results_)[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']]
best_LRdf = best_LRdf.sort_values(by='rank_test_score', ascending=True)
best_LRdf = best_LRdf.reset_index(drop=True)
best_LRdf.head()

,rank_test_score,mean_test_score,std_test_score,params
0,1,0.841667,0.205649,"{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', '..."
1,2,0.816667,0.229129,"{'C': 0.1, 'max_iter': 200, 'penalty': 'l2', '..."
2,2,0.816667,0.229129,"{'C': 0.1, 'max_iter': 400, 'penalty': 'l2', '..."
3,2,0.816667,0.229129,"{'C': 0.1, 'max_iter': 400, 'penalty': 'l2', '..."
4,2,0.816667,0.229129,"{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', '..."


In [38]:
# examine the first result
print("**examine the first result","\n")

print(best_LR.cv_results_['params'][0])
print(best_LR.cv_results_['mean_test_score'][0])

# print the array of mean scores only
print("\n","**print the array of mean scores only","\n")

grid_mean_scores = best_LR.cv_results_['mean_test_score']
print(grid_mean_scores)

# examine the best model
print("\n","**examine the best model","\n")

print(best_LR.best_score_)
print(best_LR.best_params_)
print(best_LR.best_estimator_)

**examine the first result 

{'C': 0.001, 'max_iter': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
nan

 **print the array of mean scores only 

[       nan        nan 0.35              nan 0.375      0.63333333
 0.63333333 0.46666667 0.55833333 0.55833333        nan        nan
 0.35              nan 0.375      0.63333333 0.63333333 0.46666667
 0.55833333 0.55833333        nan        nan 0.35              nan
 0.375      0.63333333 0.63333333 0.46666667 0.55833333 0.55833333
        nan        nan 0.35              nan 0.375      0.63333333
 0.63333333 0.46666667 0.55833333 0.55833333        nan        nan
 0.35              nan 0.35       0.78333333 0.78333333 0.625
 0.625      0.625             nan        nan 0.35              nan
 0.35       0.78333333 0.78333333 0.625      0.625      0.625
        nan        nan 0.35              nan 0.35       0.78333333
 0.78333333 0.625      0.625      0.625             nan        nan
 0.35              nan 0.35       0.78333333 0.78333333 0.625

In [39]:
#Print the tured parameters and score
print("Tuned Parameters: {}".format(best_LR.best_params_))
print("Best score is {}".format(best_LR.best_score_))

Tuned Parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score is 0.8416666666666668


In [41]:
best_params_ =  best_LR.best_params_
C_ = best_params_['C'] 
max_iter_ = best_params_['max_iter'] 
penalty_ = best_params_['penalty']   
solver_ = best_params_['solver']

> ## 🚀 Fit Model and setting parameters

In [42]:
## **- train Fit Model XGBOOST
Logistic = LogisticRegression(random_state = 1, C=C_, max_iter=max_iter_, penalty=penalty_, solver=solver_, multi_class='multinomial')
LogisticModel = Logistic.fit(X_train, y_train) # train XGBOOST model
LogisticModel

LogisticRegression(C=0.1, multi_class='multinomial', random_state=1)

In [43]:
### Testing Check Parameter
print(LogisticModel.get_params())

{'C': 0.1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'l2', 'random_state': 1, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


> ## #️⃣ Save model

In [44]:
numclass = 3

In [45]:
#Save Model
import os
import pickle
import imageio

save_pathimg = f'/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/_{numclass}Classes/fold{fold}'
##**Mkdir Directory 
os.makedirs(save_pathimg, exist_ok=True)     
filename = f"Logis_Model_Blood{numclass}Class_fold{fold}.pkl"
Model2Save =  f"{save_pathimg}/{filename}"
print(f"[INFO]: Done!! Save Model as : {Model2Save}")

with open(Model2Save, 'wb') as file:
    pickle.dump(LogisticModel, file)

[INFO]: Done!! Save Model as : /media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/_3Classes/fold3/Logis_Model_Blood3Class_fold3.pkl
